The first few cells set up phonetisaurus; they are adapted from the instructions in the git README.

In [1]:
%%capture
!apt-get -y install git g++ autoconf-archive make libtool
# Python bindings
!apt-get -y install python-setuptools python-dev
!apt-get -y install gfortran

In [2]:
%%capture
%cd /tmp
!wget http://www.openfst.org/twiki/pub/FST/FstDownload/openfst-1.6.2.tar.gz
!tar -xvzf openfst-1.6.2.tar.gz
%cd openfst-1.6.2
!./configure --enable-static --enable-shared --enable-far --enable-ngram-fsts
!make -j 4
# Now wait a while...
!make install

In [3]:
import os
ldlibpath = os.environ['LD_LIBRARY_PATH']
newld = f'{ldlibpath}:/usr/local/lib:/usr/local/lib/fst'
os.environ['LD_LIBRARY_PATH']=newld
%env LD_LIBRARY_PATH

'/opt/conda/lib:/usr/local/lib:/usr/local/lib/fst'

In [4]:
%%capture
%cd /tmp
!git clone https://github.com/AdolfVonKleist/Phonetisaurus.git
%cd Phonetisaurus
!./configure
!make
!make install

In [5]:
%cd /tmp/

/tmp


We also need MITLM

In [6]:
%%capture
!git clone https://github.com/mitlm/mitlm
%cd mitlm
!autoreconf -i
!./configure
!make
!make install

In [7]:
%cd /kaggle/working

/kaggle/working


The TIMIT dictionary is relatively clean, so there are only a few small changes that are needed for phonetisaurus.

In [8]:
!cat ../input/darpa-timit-acousticphonetic-continuous-speech/TIMITDIC.TXT|grep -v '^;'|tr -d '/'|sed -e 's/  */ /g;s/~adj//;s/~v_past//;s/~v_pres//;s/~v//;s/~n//;' > /tmp/TIMIT.cleaned
!cat /tmp/TIMIT.cleaned | perl -pe 's/\s+/ /g; s/^\s+//; s/\s+$//; @_ = split (/\s+/); $w = shift (@_); $_ = $w."\t".join (" ", @_)."\n";' > /tmp/TIMIT.clean

In [9]:
!phonetisaurus-align --input=/tmp/TIMIT.clean --ofile=TIMIT.clean.corpus --seq1_del=false
# Train an n-gram model (5s-10s):
!estimate-ngram -o 8 -t TIMIT.clean.corpus -wl timit.arpa
# Convert to OpenFst format (10s-20s):
!phonetisaurus-arpa2wfst --lm=timit.arpa --ofile=timit.fst

GitRevision: 0.9.1
Loading input file: /tmp/TIMIT.clean
Alignment failed: x
Starting EM...
Finished first iter...
Iteration: 1 Change: 2.70318
Iteration: 2 Change: 0.0603504
Iteration: 3 Change: 0.0425539
Iteration: 4 Change: 0.0206814
Iteration: 5 Change: 0.0114756
Iteration: 6 Change: 0.00711536
Iteration: 7 Change: 0.0042429
Iteration: 8 Change: 0.00297546
Iteration: 9 Change: 0.00223923
Iteration: 10 Change: 0.00151825
Iteration: 11 Change: 0.00115204
Last iteration: 
0.001	Loading corpus TIMIT.clean.corpus...
0.035	Smoothing[1] = ModKN
0.035	Smoothing[2] = ModKN
0.035	Smoothing[3] = ModKN
0.035	Smoothing[4] = ModKN
0.035	Smoothing[5] = ModKN
0.035	Smoothing[6] = ModKN
0.035	Smoothing[7] = ModKN
0.035	Smoothing[8] = ModKN
0.035	Set smoothing algorithms...
0.035	Y 6.063492e-01
0.035	Y 6.304450e-01
0.035	Y 7.305669e-01
0.035	Y 7.950124e-01
0.036	Y 8.524463e-01
0.036	Y 9.033717e-01
0.036	Y 9.355036e-01
0.037	Y 9.092702e-01
0.037	Estimating full n-gram model...
0.039	Saving LM to timit

In [10]:
#!phonetisaurus-apply --model timit.fst --word_list <wordlist>